In [ ]:
!pip install evds

from evds import evdsAPI
import pandas as pd
import numpy as np

In [ ]:
evds = evdsAPI('kvlUQV03Zp')

term = '2023-09'

term_year = term[:4]
term_month = term[-2:]
term_end_day = pd.Period('{}-01-{}'.format(term_month, term_year), freq='M').end_time.day

start_date_str = '01-{}-{}'.format(term_month, term_year)
end_date_str = '{}-{}-{}'.format(term_end_day, term_month, term_year)

data_list = ['TP.DK.AUD.A.YTL', 'TP.DK.CAD.A.YTL', 'TP.DK.CHF.A.YTL', 'TP.DK.DKK.A.YTL', 'TP.DK.EUR.A.YTL', 'TP.DK.GBP.A.YTL', 'TP.DK.JPY.A.YTL', 'TP.DK.KWD.A.YTL', 'TP.DK.NOK.A.YTL', 'TP.DK.SAR.A.YTL', 'TP.DK.SEK.A.YTL', 'TP.DK.USD.A.YTL']

df_evds_ort = evds.get_data(data_list, startdate = start_date_str, enddate = end_date_str, aggregation_types='avg', frequency = 5)
df_evds_son = evds.get_data(data_list, startdate = start_date_str, enddate = end_date_str)


df_ort = pd.DataFrame()
df_ort['Döviz'] = data_list
df_ort['Döviz'] = df_ort['Döviz'].apply(lambda x: x[6:9])
df_ort[term] = list(df_evds_ort.iloc[0, 1:])
df_ort = df_ort.sort_values('Döviz').set_index('Döviz')
df_ort['Ortalama'] = np.round(df_ort[term] * 1_000_000).astype(int).astype(str)
df_ort.loc['JPY', 'Ortalama'] = '{:.2f}'.format(np.round(df_ort.loc['JPY', term] * 10_000, 2))
df_ort = df_ort.reset_index()

if df_evds_son.iloc[-1, 1] > 0:
  df_son_fix = df_evds_son.copy()
  df_son_fix = df_son_fix[df_son_fix.iloc[:,1].notnull()].sort_values('Tarih', ascending = False).head(1)
else:
  holiday = True
  next_term_year = int(term_year)
  next_term_month = int(term_month) + 1

  if next_term_month > 12:
    next_term_month -= 12
    next_term_year += 1

  next_term_year = str(next_term_year)
  next_term_month = str(next_term_month).zfill(2)
  next_day = 0

  while(holiday):
    next_day += 1
    next_date = '{}-{}-{}'.format(str(next_day).zfill(2), next_term_month, next_term_year)

    try:
      df_next = evds.get_data(data_list, startdate = next_date, enddate = next_date)
    except:
      print(next_date + 'için veri alınamadı')

    if df_next.iloc[-1, 1] > 0:
      holiday = False
      df_son_fix = df_next.copy()

df_son = pd.DataFrame()
df_son['Döviz'] = data_list
df_son['Döviz'] = df_son['Döviz'].apply(lambda x: x[6:9])
df_son[term] = list(df_son_fix.iloc[0, 1:])
df_son = df_son.sort_values('Döviz').set_index('Döviz')
df_son['Yıl Sonu'] = np.round(df_son[term] * 1_000_000).astype(int).astype(str)
df_son.loc['JPY', 'Yıl Sonu'] = '{:.2f}'.format(np.round(df_son.loc['JPY', term] * 10_000, 2))
df_son = df_son.reset_index()


df_na = pd.DataFrame({'': [''] * len(df_ort)})
final_df_list = []
final_df_list.append(df_ort)
for _ in range(3):
  final_df_list.append(df_na)
final_df_list.append(df_son)
df_final = pd.concat(final_df_list, axis=1)
df_final.index = [''] * len(df_final)

02-10-2023


# Ortalama & Yıl Sonu

In [ ]:
df_final

,Döviz,2023-09,Ortalama,,,,Döviz,2023-09,Yıl Sonu
,AUD,17.266938,17266938,,,,AUD,17.7107,17710700
,CAD,19.868205,19868205,,,,CAD,20.3435,20343500
,CHF,29.973567,29973567,,,,CHF,29.9837,29983700
,DKK,3.859367,3859367,,,,DKK,3.8871,3887100
,EUR,28.820471,28820471,,,,EUR,29.0305,29030500
,GBP,33.407757,33407757,,,,GBP,33.4816,33481600
,JPY,18.215967,182159.67,,,,JPY,18.3267,183267.00
,KWD,86.773300,86773300,,,,KWD,88.1206,88120600
,NOK,2.506319,2506319,,,,NOK,2.5727,2572700
,SAR,7.182343,7182343,,,,SAR,7.2996,7299600
